# Amazon Web Scraper Project

In [6]:
# IMPORT LIB

from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
import datetime
import smtplib #for sending emails
import csv
import datetime
import os
import csv
import smtplib
import settings as s
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [7]:
# CONNECTION

def connect(URL):
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36", 
            "Accept-Encoding":"gzip, deflate, br", 
            "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7", 
            "DNT":"1",
            "Connection":"close", 
            "Upgrade-Insecure-Requests":"1"
            }
    page = requests.get(URL,headers=headers)

    soup1 = BeautifulSoup(page.content, "html.parser")
    return BeautifulSoup(soup1.prettify(), 'html.parser')

# OPEN TITLE
def find_title(bs):
    title = bs.find(id="productTitle").get_text().strip()
    return title

# PRICE : FIND FIRST SPAN WHERE class = "a-offscreen"
def find_price(bs):
    price = bs.find("span", {"class": "a-offscreen"}).get_text().strip().replace('$','')
    return price

# SAVE DATA TO FILE
def write_data(file, title, price):

    today = datetime.date.today()
    header = ['Title','Price','Date']
    data = [title, price, today]
    type(data)
    if os.path.isfile(file):
        with open(file,'a+', newline = '', encoding = 'UTF8') as f:
            writer = csv.writer(f)
            writer.writerow(data)
    else:
        with open(file,'w', newline = '', encoding = 'UTF8') as f:
            writer = csv.writer(f)
            writer.writerow(header)
            writer.writerow(data)
    return True


#CHECK PRICE
def check_price(price, min_price):
    if (float(price) < float(min_price)):
        send_massage()
        return True
    return False

# SEND MESSAGE
def send_massage():
    mail_content = "hh"
    sender_address = s.sender_address
    sender_pass = s.sender_psw
    receiver_address = s.receiver_address
    #Setup the MIME
    message = MIMEMultipart()
    message['From'] = sender_address
    message['To'] = receiver_address
    message['Subject'] = 'A test mail sent by Python. It has an attachment.'   #The subject line
    #The body and the attachments for the mail
    message.attach(MIMEText(mail_content, 'plain'))
    #Create SMTP session for sending the mail
    session = smtplib.SMTP('smtp.office365.com', 587) #use gmail with port
    session.starttls() #enable security
    session.login(sender_address, sender_pass) #login with mail_id and password
    text = message.as_string()
    session.sendmail(sender_address, receiver_address, text)
    session.quit()
    print('Mail Sent')
    return True



In [9]:
# MAIN

file = 'files/AmazonWebScraper.csv'
URL = 'https://www.amazon.com/Amazon-Essentials-Womens-Skirt-Camel/dp/B08JH893DJ/ref=sr_1_2?crid=2EPWJXDHO814T&keywords=skirt&qid=1686339482&sprefix=skirt%2Caps%2C213&sr=8-2&th=1&psc=1'
low_price = 25

bs = connect(URL)
price = find_price(bs)
title = find_title(bs)

if write_data(file, title, price):
    pd.read_csv(file)

if check_price(price, low_price):
    print('price of',title,'lower than', low_price)
else:
    print('price of',title,'higher than', low_price)


price of Amazon Essentials Women's Pull-On Knit Midi Skirt (Available in Plus Size) higher than 25


In [ ]:
# FIND SPAN WHERE class = "a-offscreen"
bs.find_all("span", {"class": "a-offscreen"})

In [ ]:
# LOOP FOR CHECK PRICE

def loop_price():
    while(True):
        check_price()
        time.sleep(5)